<a href="https://colab.research.google.com/github/SophiaU/Git_DoD_Remittance/blob/main/Old_File_Format_Data_Dump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install and import pygsheets
!pip install pygsheets
import pygsheets

#  Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# connect google sheets
import gspread
# import pygsheets
gc = gspread.authorize(GoogleCredentials.get_application_default())

# import pandas
import pandas as pd
from functools import reduce


from google.colab import files
uploaded = files.upload()

import json
from google.oauth2 import service_account

with open('service_account.json') as source:
    info = json.load(source)
credentials = service_account.Credentials.from_service_account_info(info)

client = pygsheets.authorize(service_account_file='service_account.json')


Saving service_account.json to service_account.json


Combine Performance File

In [ ]:
# Append all files in folder into a listframe

listframe = []

weekname = 'Nov 18'

fileid = '1AFwJmZoFVyaB4y2uWF9ZwcufVEI3d3eg'

# lagos = "(not title contains '(30)' and not title contains '(20)' and not title contains '(10)' and not title contains '(9)' and not title contains '(7)' and not title contains '(6)')"
lagos = "(title contains '(12)'  or title contains '(13)' or title contains '(14)')"
ghana = "(title contains '(6)' or title contains '(9)')"
# sa = "(title contains '(7)' or title contains '(10)' or title contains '(30)' or title contains '(20)')"
sa = "(title contains '(31)' or title contains '(32)')"

folder = drive.ListFile({'q': "title contains '"+weekname+"' and "+sa+" and '"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()

for file in folder:
  print(file['title'])

  # download
  fileDownloaded = drive.CreateFile({'id':file['id']})

  # load
  fileDownloaded.GetContentFile('performance.csv')

  # read
  df = pd.read_csv('performance.csv', delimiter=',' )
  

  # append
  listframe.append(df)

# combine all files in the list into a dataframe
dayperformancefile = pd.concat(listframe)

performancefile = dayperformancefile
performancefile['Name'] = pd.Series(performancefile['Name']).str.title()

performancefile.head(400)




driver_performance Nov 18(32).csv
driver_performance Nov 18(31).csv


,Name,Net Fares,Per trip,Per hour online,Per km on trip,Trips,Hours online,Trips per hour,Distance per trip,Ratings,Lifetime rating,Acceptance rate,Driver cancellation rate
0,Zibusiso Ndlovu,1295.50,71.97,78.04,9.10,18,16h 36m,1.1,12.73 KM,5,4.92,58.82%,10.00%
1,Mandla Maphanga,1069.50,48.61,76.39,11.23,22,14h 0m,1.6,6.97 KM,5,4.84,56.60%,16.67%
2,Khathutshelo Nemaguvhuni,1044.00,65.25,90.78,10.13,16,11h 30m,1.4,10.37 KM,5,4.98,33.96%,11.11%
3,Sphamandla Mdletshe,1010.00,67.33,77.10,9.50,15,13h 6m,1.1,11.40 KM,5,4.90,46.94%,26.09%
4,Anderson Dube,987.25,70.52,80.92,9.08,14,12h 12m,1.1,12.50 KM,5,4.92,57.69%,0.00%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,Evance Poto,269.25,89.75,67.31,9.69,3,4h 0m,0.8,14.90 KM,—,4.81,100.00%,0.00%
36,Daniel Dube,246.00,49.20,37.85,10.12,5,6h 30m,0.8,7.82 KM,4.5,4.86,50.00%,16.67%
37,Ayanda Khumalo,218.25,72.75,55.96,8.79,3,3h 54m,0.8,13.32 KM,5,4.93,36.84%,42.86%
38,Adroyth Mathebula,189.00,94.50,30.48,10.04,2,6h 12m,0.3,15.15 KM,—,4.87,40.00%,0.00%


Combine All Drivers

In [ ]:
# Append all files in folder into a listframe

alldriverslistframe = []

fileid = '1BrPHK2OlLCd1LZAKxbUYmzspdCnySPkd'

folder = drive.ListFile({'q': "title contains '"+weekname+"' and "+sa+" and'"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()

# 'q': " and (not title contains '(30)' and not title contains '(20)' and not title contains '(10)' and not title contains '(9)' and not title contains '(7)' and not title contains '(6)') and'"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "})
# Accra - 'q': "title contains '"+weekname+"' and (title contains '(6)' or title contains '(9)') and'"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()
# JBurg - 'q': "title contains '"+weekname+"' and (title contains '(7)' or title contains '(10)' or title contains '(30)') and'"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()
# CPT - 'q': "title contains '"+weekname+"' and (title contains '(20)' ) and'"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()
# folder = drive.ListFile({'q': "(title contains '"+weekname0+"' or title contains '"+weekname1+"' or title contains '"+weekname2+"' or title contains '"+weekname3+"' or title contains '"+weekname4+"' or title contains '"+weekname5+"' or title contains '"+weekname6+"' ) and not title contains '(30)' and not title contains '(20)' and not title contains '(10)' and'"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()


for file in folder:
  print(file['title'])

  # download
  fileDownloaded = drive.CreateFile({'id':file['id']})

  # load
  fileDownloaded.GetContentFile('alldrivers.csv')

  # read
  df = pd.read_csv('alldrivers.csv', delimiter=',' )

  df = df.rename({'First name':'First Name','Last name':'Last Name'}, axis=1)

  # print(df.columns)
  # print(len(df.columns))

  # append
  alldriverslistframe.append(df)

# combine all files in the list into a dataframe
combinedAlldrivers = pd.concat(alldriverslistframe)

combinedAlldrivers['Name'] = combinedAlldrivers['First Name'] + " " + combinedAlldrivers['Last Name']

# Pick only Full name, Uber ID and Uber Fees
alldriversfile = combinedAlldrivers[['Driver ID','Name','Uber Fee']]

alldriversfile['Name'] = pd.Series(alldriversfile['Name']).str.title()

alldriversfile.head(10)



All Drivers Nov 18(32).csv
All Drivers Nov 18(31).csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Driver ID,Name,Uber Fee
0,68409f1b-3d8e-45c9-8b84-cdeba53b817d,Jacob Mogotsi,-127.00
1,61e00b21-69ff-4a85-b781-8590e63ea599,Keitumetse Donald Mahalefa,-233.00
2,cfb27c23-38f5-470f-ac16-4bb870ec54ea,Sphamandla Mdletshe,-316.00
3,fe65e2d6-b36e-46c0-ab38-3c777eaf1fa4,Wandaboy Malele,-177.50
4,a504d409-330e-40d0-b9bb-0d0dfc6c5231,Mpilo Ngwenya,-8.25
5,2ed77eab-8105-4904-8caa-cfc63b29ba1d,Nhlalwenhle Mzaca,-141.50
6,3c1da8cf-5384-435f-b0b7-26a96bd2af06,Freedom Ncube,-181.00
7,9bd96855-406b-4f66-91a1-72641fc25978,Thembinkosi Manzini,-284.25
8,9a134400-93f5-4953-a012-8a8212b15f83,Hendrick Morodi,-217.50
9,97bb38f5-86e9-4748-a434-0b1bc29e918f,Elijah Moyo,-264.25


Combine Performance and All Drivers to get Gross Fare

In [ ]:
# Combine Performance and All Drivers files to get Gross fares
# performancefile
# alldriversfile

  # join elements of the final files
dfs = [performancefile,alldriversfile]

  # Join on Name 
reconfile = reduce(lambda left,right: pd.merge(left,right,how='left', on='Name'), dfs)

reconfile['Date'] = weekname
reconfile['GrossFares'] = reconfile['Net Fares'] - reconfile['Uber Fee']

 # wrangle reconfile
reconfile = reconfile.fillna(0)

reconfile = reconfile[['Date','Name', 'Net Fares', 'Per trip', 'Per hour online', 'Per km on trip',
        'Trips', 'Hours online', 'Trips per hour', 'Distance per trip',
        'Ratings', 'Lifetime rating', 'Acceptance rate',
        'Driver cancellation rate', 'GrossFares','Driver ID']]

reconfile.head(10)


,Date,Name,Net Fares,Per trip,Per hour online,Per km on trip,Trips,Hours online,Trips per hour,Distance per trip,Ratings,Lifetime rating,Acceptance rate,Driver cancellation rate,GrossFares,Driver ID
0,Nov 18,Zibusiso Ndlovu,1295.50,71.97,78.04,9.10,18,16h 36m,1.1,12.73 KM,5,4.92,58.82%,10.00%,1726.75,19aea129-02db-4c7b-8f62-4bf368eca388
1,Nov 18,Mandla Maphanga,1069.50,48.61,76.39,11.23,22,14h 0m,1.6,6.97 KM,5,4.84,56.60%,16.67%,1428.50,c5582b88-08d1-445a-a47a-721e58a1ba0d
2,Nov 18,Khathutshelo Nemaguvhuni,1044.00,65.25,90.78,10.13,16,11h 30m,1.4,10.37 KM,5,4.98,33.96%,11.11%,1367.00,93abb2d9-7174-4cef-b7e0-4331aa0b37a1
3,Nov 18,Sphamandla Mdletshe,1010.00,67.33,77.10,9.50,15,13h 6m,1.1,11.40 KM,5,4.90,46.94%,26.09%,1326.00,cfb27c23-38f5-470f-ac16-4bb870ec54ea
4,Nov 18,Anderson Dube,987.25,70.52,80.92,9.08,14,12h 12m,1.1,12.50 KM,5,4.92,57.69%,0.00%,1303.00,563e0293-415e-4a8c-bbfd-46e31a253264
5,Nov 18,Mpho Radzilani,924.75,57.80,75.18,10.06,16,12h 18m,1.3,9.25 KM,5,4.93,50.00%,5.56%,1233.00,8e3044d0-d0cc-454c-8087-8e0d20832d84
6,Nov 18,Thembinkosi Manzini,852.75,77.52,59.63,9.22,11,14h 18m,0.8,13.54 KM,5,4.89,28.95%,0.00%,1137.00,9bd96855-406b-4f66-91a1-72641fc25978
7,Nov 18,Gundo Simba,808.75,53.92,58.61,10.33,15,13h 48m,1.1,8.40 KM,5,4.90,43.24%,6.25%,1075.00,7e11a069-82d4-4f36-a560-82636120a783
8,Nov 18,Elijah Moyo,792.75,66.06,53.93,9.98,12,14h 42m,0.8,10.65 KM,5,4.93,38.24%,0.00%,1057.00,97bb38f5-86e9-4748-a434-0b1bc29e918f
9,Nov 18,Mkhululi Mncube,740.75,46.30,56.55,11.71,16,13h 6m,1.2,6.36 KM,5,4.97,48.57%,0.00%,983.50,8b8ee396-0c93-49fd-bd8c-d978ca7dbe92


Write reconciled files into Lagos Data Dump

In [ ]:
dataSheetLag = client.open_by_key('1Q3BJcXwRncZiiB3GAYw1CNN-RWOWjt4AyUHxjs_Kztg')
dayreconfileLag = dataSheetLag.worksheet_by_title("oldFormat")
lastrow = len(dayreconfileLag.get_values('A1','A50000')) +1

dayreconfileLag.set_dataframe(reconfile,(lastrow,3),copy_head=False,nan='')

Write to SA Data Dump

In [ ]:
dataSheetSA = client.open_by_key('1YDi41F705U9R4MBda0tILrkxR0uM8b9iX-MK5yombQQ')
dayreconfileSA = dataSheetSA.worksheet_by_title("oldFormat")
lastrow = len(dayreconfileSA.get_values('A1','A20000')) +1

dayreconfileSA.set_dataframe(reconfile,(lastrow,4),copy_head=False,nan='')

In [ ]:
# # export to csv

# dailyReconfile_csv = reconfile.to_csv( "DailyReconFile_Nov14th_SA.csv", index=False, encoding='utf-8-sig')
# dailyReconfile_csv = reconfile.to_csv( "DailyReconFile_Nov14th_Accra.csv", index=False, encoding='utf-8-sig')
dailyReconfile_csv = reconfile.to_csv( "DailyReconFile_Nov14th_Lagos.csv", index=False, encoding='utf-8-sig')


file2 = drive.CreateFile({'parents': [{'id': '1xZNnk4qko7I1S5--BdrF5LlSAr3EF3fY'}]})
file2.SetContentFile("DailyReconFile_Nov14th_Lagos.csv")
file2.Upload()


Search for a Record

In [ ]:
# NameCheck = performancefile1.query('driverUUID == "de90e988-5c1d-4ee7-ae7d-0a1418cdd111"')
# NameCheck = performancefile.query('Name == "Abdulazeez Oluwadamilare Isiaka"')     # Isiaq Jamiu Akanbi
NameCheck = combinedAlldrivers.query('Name == "Adetunji Kehinde Atobatele"')     # Isiaq Jamiu Akanbi
NameCheck

,Driver ID,First Name,Last Name,Total,Trips,Quest Promotion,Fare,Wait Time,Surge,Cash collection,Uber Fee,Other Promotions,Tip,Adjusted Fare,Toll,Lagos Airport Surcharge,Uber Fee Collection,Name
6,8fd3e159-f58b-41c0-ae24-13bee43a9299,Adetunji Kehinde,Atobatele,1314.25,2,NaN,2212.0,NaN,607.0,-800.0,-704.75,NaN,NaN,NaN,NaN,NaN,NaN,Adetunji Kehinde Atobatele
